## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt


import gmaps
import sys
sys.path.append('..')
from config import weather_api_key
from config import g_key


# Configure gmaps API key
gmaps.configure(api_key=g_key)

from citipy import citipy

import requests
import time

In [3]:
# 1. Import the WeatherPy_database.csv file. 
WeatherPy_database_df = pd.read_csv("../Weather_Database/WeatherPy_database.csv")
WeatherPy_database_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,weather Description
0,0,Chokurdakh,RU,70.6333,147.9167,49.15,48,9,8.12,clear sky
1,1,Sao Jose Da Coroa Grande,BR,-8.8978,-35.1478,74.08,77,33,9.82,scattered clouds
2,2,Fortuna,US,40.5982,-124.1573,66.92,79,100,11.50,overcast clouds
3,3,Cape Town,ZA,-33.9258,18.4232,54.37,75,4,5.01,clear sky
4,4,Aswan,EG,24.0934,32.9070,87.28,23,3,11.48,clear sky


In [4]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 75
What is the maximum temperature you would like for your trip? 90


In [5]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.

preferred_cities_df = WeatherPy_database_df.loc[(WeatherPy_database_df["Max Temp"] <= max_temp) & \
                                       (WeatherPy_database_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,weather Description
4,4,Aswan,EG,24.0934,32.9070,87.28,23,3,11.48,clear sky
13,13,Komsomolskiy,UZ,40.4272,71.7189,77.72,41,43,6.91,scattered clouds
16,16,Butaritari,KI,3.0707,172.7902,81.55,78,48,14.58,light rain
17,17,Ponta Delgada,PT,37.7333,-25.6667,75.92,88,40,9.22,scattered clouds
20,20,Primero De Enero,CU,21.9469,-78.4283,83.35,70,71,6.26,broken clouds
21,21,Jibuti,DJ,11.5000,42.5000,85.60,54,76,3.47,broken clouds
26,26,Dingle,PH,10.9995,122.6711,76.19,93,97,4.88,light rain
31,31,Bethel,US,41.3712,-73.4140,79.03,51,74,5.93,broken clouds
32,32,Kavieng,PG,-2.5744,150.7967,82.44,69,74,7.09,broken clouds
33,33,Luwuk,ID,-0.9516,122.7875,77.45,83,99,5.70,overcast clouds


In [6]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.count()

City_ID                117
City                   117
Country                117
Lat                    117
Lng                    117
Max Temp               117
Humidity               117
Cloudiness             117
Wind Speed             117
weather Description    117
dtype: int64

In [7]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
preferred_cities_df.dropna()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,weather Description
4,4,Aswan,EG,24.0934,32.9070,87.28,23,3,11.48,clear sky
13,13,Komsomolskiy,UZ,40.4272,71.7189,77.72,41,43,6.91,scattered clouds
16,16,Butaritari,KI,3.0707,172.7902,81.55,78,48,14.58,light rain
17,17,Ponta Delgada,PT,37.7333,-25.6667,75.92,88,40,9.22,scattered clouds
20,20,Primero De Enero,CU,21.9469,-78.4283,83.35,70,71,6.26,broken clouds
...,...,...,...,...,...,...,...,...,...,...
349,349,Roma,US,43.2128,-75.4557,78.94,50,0,3.44,clear sky
352,352,Minjur,IN,13.2667,80.2667,84.22,70,100,7.36,overcast clouds
354,354,Methoni,GR,36.8197,21.7049,76.21,65,0,17.43,clear sky
357,357,Micheweni,TZ,-4.9667,39.8333,75.33,75,77,9.46,broken clouds


In [8]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = preferred_cities_df[["City", "Country", "Max Temp", "weather Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,weather Description,Lat,Lng,Hotel Name
4,Aswan,EG,87.28,clear sky,24.0934,32.9070,
13,Komsomolskiy,UZ,77.72,scattered clouds,40.4272,71.7189,
16,Butaritari,KI,81.55,light rain,3.0707,172.7902,
17,Ponta Delgada,PT,75.92,scattered clouds,37.7333,-25.6667,
20,Primero De Enero,CU,83.35,broken clouds,21.9469,-78.4283,
21,Jibuti,DJ,85.60,broken clouds,11.5000,42.5000,
26,Dingle,PH,76.19,light rain,10.9995,122.6711,
31,Bethel,US,79.03,broken clouds,41.3712,-73.4140,
32,Kavieng,PG,82.44,broken clouds,-2.5744,150.7967,
33,Luwuk,ID,77.45,overcast clouds,-0.9516,122.7875,


In [9]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    params["location"] = f"{lat},{lng}"
      
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json" 

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError): 
        print("Hotel not found... skipping.") 

        

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [10]:
hotel_df.head(10)

,City,Country,Max Temp,weather Description,Lat,Lng,Hotel Name
4,Aswan,EG,87.28,clear sky,24.0934,32.9070,Nuba Nile
13,Komsomolskiy,UZ,77.72,scattered clouds,40.4272,71.7189,Ikathouse
16,Butaritari,KI,81.55,light rain,3.0707,172.7902,Isles Sunset Lodge
17,Ponta Delgada,PT,75.92,scattered clouds,37.7333,-25.6667,Vila Nova Hotel
20,Primero De Enero,CU,83.35,broken clouds,21.9469,-78.4283,Motel Dos Palmas (Maria Machete)
21,Jibuti,DJ,85.60,broken clouds,11.5000,42.5000,Orchid Main Camp Goubet
26,Dingle,PH,76.19,light rain,10.9995,122.6711,SEGAYA PROPERTY
31,Bethel,US,79.03,broken clouds,41.3712,-73.4140,Courtyard by Marriott Danbury
32,Kavieng,PG,82.44,broken clouds,-2.5744,150.7967,Nusa Island Retreat
33,Luwuk,ID,77.45,overcast clouds,-0.9516,122.7875,Hotel Santika Luwuk


In [11]:
# 7. Drop the rows where there is no Hotel Name.
clean_hotel_df = hotel_df[hotel_df["Hotel Name"] != ""]
clean_hotel_df.head(10)

,City,Country,Max Temp,weather Description,Lat,Lng,Hotel Name
4,Aswan,EG,87.28,clear sky,24.0934,32.9070,Nuba Nile
13,Komsomolskiy,UZ,77.72,scattered clouds,40.4272,71.7189,Ikathouse
16,Butaritari,KI,81.55,light rain,3.0707,172.7902,Isles Sunset Lodge
17,Ponta Delgada,PT,75.92,scattered clouds,37.7333,-25.6667,Vila Nova Hotel
20,Primero De Enero,CU,83.35,broken clouds,21.9469,-78.4283,Motel Dos Palmas (Maria Machete)
21,Jibuti,DJ,85.60,broken clouds,11.5000,42.5000,Orchid Main Camp Goubet
26,Dingle,PH,76.19,light rain,10.9995,122.6711,SEGAYA PROPERTY
31,Bethel,US,79.03,broken clouds,41.3712,-73.4140,Courtyard by Marriott Danbury
32,Kavieng,PG,82.44,broken clouds,-2.5744,150.7967,Nusa Island Retreat
33,Luwuk,ID,77.45,overcast clouds,-0.9516,122.7875,Hotel Santika Luwuk


In [13]:
# 8a. Create the output File (CSV)
output_data_file = "WeatherPy_vacation.csv"
# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [14]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Weather</dt><dd>{weather Description} and {Max Temp} °F</dd>
</dl>
"""
# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [15]:
# 11a. Add a marker layer for each city to the map. 
locations = clean_hotel_df[["Lat", "Lng"]]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)

marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)

# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))

In [1]:
#savefig
("../Vacation_Search/WeatherPy_vacation_map.png")

'../Vacation_Search/WeatherPy_vacation_map.png'